In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football/aerial-photo-of-soccer-field jpg.jpg
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (26).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (9).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (13).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (28).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (19).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (21).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (27).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (6).mp4
/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/te

In [2]:
'''import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load tracking data (replace with your actual file)
df = pd.read_csv('tracking_data.csv')

# Filter one frame for visualization (example)
frame_data = df[df['frame'] == 100]  # Pick any frame

# Create pitch
fig, ax = plt.subplots(figsize=(12, 8))
pitch = patches.Rectangle((0, 0), 105, 68, linewidth=2, edgecolor='black', facecolor='green', alpha=0.5)
ax.add_patch(pitch)

# Draw pitch markings
plt.plot([52.5, 52.5], [0, 68], color='white', linestyle='--')  # Halfway line
plt.plot([16.5, 16.5], [13.84, 54.16], color='white')  # Penalty area
plt.plot([88.5, 88.5], [13.84, 54.16], color='white')

# Plot players
for _, row in frame_data.iterrows():
    color = 'red' if row['team_id'] == 1 else 'blue'  # Adjust team_id as per your data
    plt.scatter(row['x'], row['y'], color=color, s=200, edgecolors='white')
    plt.text(row['x'], row['y'], str(row['player_id']), color='white', ha='center', va='center')

plt.xlim(0, 105)
plt.ylim(0, 68)
plt.axis('off')
plt.title('2D Player Position Map')
plt.show()'''

"import pandas as pd\nimport matplotlib.pyplot as plt\nimport matplotlib.patches as patches\n\n# Load tracking data (replace with your actual file)\ndf = pd.read_csv('tracking_data.csv')\n\n# Filter one frame for visualization (example)\nframe_data = df[df['frame'] == 100]  # Pick any frame\n\n# Create pitch\nfig, ax = plt.subplots(figsize=(12, 8))\npitch = patches.Rectangle((0, 0), 105, 68, linewidth=2, edgecolor='black', facecolor='green', alpha=0.5)\nax.add_patch(pitch)\n\n# Draw pitch markings\nplt.plot([52.5, 52.5], [0, 68], color='white', linestyle='--')  # Halfway line\nplt.plot([16.5, 16.5], [13.84, 54.16], color='white')  # Penalty area\nplt.plot([88.5, 88.5], [13.84, 54.16], color='white')\n\n# Plot players\nfor _, row in frame_data.iterrows():\n    color = 'red' if row['team_id'] == 1 else 'blue'  # Adjust team_id as per your data\n    plt.scatter(row['x'], row['y'], color=color, s=200, edgecolors='white')\n    plt.text(row['x'], row['y'], str(row['player_id']), color='white

In [3]:
pip install opencv-python numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
'''import cv2
import numpy as np

# 1. Setup
video_path = "/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (1).mp4"  # Your input video
output_path = "2d_tracking_output.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = 800, 600  # Output video dimensions

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 2. Create a blank pitch image
def create_pitch():
    img = np.zeros((600, 800, 3), dtype=np.uint8)
    img[:,:] = (40, 135, 58)  # Green pitch
    
    # Draw markings (white lines)
    cv2.rectangle(img, (50, 50), (750, 550), (255,255,255), 2)  # Border
    cv2.line(img, (400, 50), (400, 550), (255,255,255), 2)      # Halfway line
    cv2.circle(img, (400, 300), 70, (255,255,255), 2)           # Center circle
    return img

# 3. Process video
fgbg = cv2.createBackgroundSubtractorMOG2()  # Motion detector

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect moving objects (players)
    fgmask = fgbg.apply(frame)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create new pitch frame
    pitch_img = create_pitch()
    
    # Draw players as dots
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            pitch_x = int(np.interp(x + w/2, [0, frame.shape[1]], [50, 750]))
            pitch_y = int(np.interp(y + h/2, [0, frame.shape[0]], [550, 50]))
            
            cv2.circle(pitch_img, (pitch_x, pitch_y), 8, (0,0,255), -1)  # Red dot
    
    # Write to output video
    out.write(pitch_img)

cap.release()
out.release()
print(f"Output saved to {output_path}")'''

'import cv2\nimport numpy as np\n\n# 1. Setup\nvideo_path = "/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (1).mp4"  # Your input video\noutput_path = "2d_tracking_output.mp4"\ncap = cv2.VideoCapture(video_path)\n\n# Get video properties\nfps = int(cap.get(cv2.CAP_PROP_FPS))\nwidth, height = 800, 600  # Output video dimensions\n\n# Create video writer\nfourcc = cv2.VideoWriter_fourcc(*\'mp4v\')\nout = cv2.VideoWriter(output_path, fourcc, fps, (width, height))\n\n# 2. Create a blank pitch image\ndef create_pitch():\n    img = np.zeros((600, 800, 3), dtype=np.uint8)\n    img[:,:] = (40, 135, 58)  # Green pitch\n    \n    # Draw markings (white lines)\n    cv2.rectangle(img, (50, 50), (750, 550), (255,255,255), 2)  # Border\n    cv2.line(img, (400, 50), (400, 550), (255,255,255), 2)      # Halfway line\n    cv2.circle(img, (400, 300), 70, (255,255,255), 2)           # Center circle\n    return img\n\n# 3. Process video\nfgbg = cv2.createB

In [5]:
'''import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle, Rectangle

# 1. SETUP - Replace this with your actual detection logic
# ----------------------------------------------------------
def detect_objects(frame):
    """Mock detection function - REPLACE WITH YOLO/ACTUAL DETECTOR"""
    height, width = frame.shape[:2]
    
    # Generate some random player positions (replace with real detections)
    num_players = 22
    players = []
    for i in range(num_players):
        team = 'A' if i < 11 else 'B'  # First 11 players are team A
        x = np.random.randint(width//4, 3*width//4)
        y = np.random.randint(height//4, 3*height//4)
        w, h = 30, 60
        players.append({
            'team': team,
            'box': [x, y, x+w, y+h],
            'confidence': np.random.uniform(0.7, 0.9)
        })
    
    # Add a ball (replace with real detection)
    ball = {
        'box': [
            width//2 + np.random.randint(-20, 20),
            height//2 + np.random.randint(-20, 20),
            width//2 + np.random.randint(-20, 20) + 10,
            height//2 + np.random.randint(-20, 20) + 10
        ],
        'confidence': np.random.uniform(0.6, 0.8)
    }
    
    return {
        'players': players,
        'ball': ball
    }

# 2. PITCH MAPPING CONFIGURATION (ADJUST THESE!)
# ----------------------------------------------------------
# Define 4 points on the video frame that correspond to pitch corners
# These should be visible in your video - you may need to manually select them
video_corners = np.float32([
    [200, 150],   # Top-left (near corner flag)
    [800, 150],   # Top-right
    [900, 550],   # Bottom-right (near penalty area)
    [100, 550]    # Bottom-left
])

# Corresponding points on our 2D pitch map (0-100 scale)
pitch_corners = np.float32([
    [0, 0],
    [100, 0],
    [100, 100],
    [0, 100]
])

# Calculate homography matrix
H, _ = cv2.findHomography(video_corners, pitch_corners)

# 3. VIDEO PROCESSING PIPELINE
# ----------------------------------------------------------
video_path = "/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (1).mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_positions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get detections (replace this with your actual detection code)
    detections = detect_objects(frame)
    
    # Process detections
    frame_data = {'players': [], 'ball': None}
    
    # Process players
    for player in detections['players']:
        if player['confidence'] < 0.5:  # Confidence threshold
            continue
            
        # Get bottom center point (assumes players stand on the ground)
        x = (player['box'][0] + player['box'][2]) / 2
        y = player['box'][3]  # Use bottom of bounding box
        
        # Map to pitch coordinates
        pitch_pos = cv2.perspectiveTransform(
            np.float32([[[x, y]]]), H
        )[0][0]
        
        frame_data['players'].append({
            'team': player['team'],
            'position': pitch_pos,
            'confidence': player['confidence']
        })
    
    # Process ball
    if detections['ball']['confidence'] > 0.3:
        x = (detections['ball']['box'][0] + detections['ball']['box'][2]) / 2
        y = (detections['ball']['box'][1] + detections['ball']['box'][3]) / 2
        
        pitch_pos = cv2.perspectiveTransform(
            np.float32([[[x, y]]]), H
        )[0][0]
        
        frame_data['ball'] = pitch_pos
    
    frame_positions.append(frame_data)

cap.release()

# 4. CREATE 2D ANIMATION
# ----------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_aspect('equal')
ax.axis('off')

# Draw pitch markings
def draw_pitch():
    # Field outline
    ax.plot([0, 100, 100, 0, 0], [0, 0, 100, 100, 0], 'white', linewidth=2)
    
    # Center line
    ax.plot([50, 50], [0, 100], 'white', linewidth=2)
    
    # Center circle
    ax.add_patch(Circle((50, 50), 10, color='white', fill=False, linewidth=2))
    
    # Penalty areas (simplified)
    ax.add_patch(Rectangle((0, 30), 20, 40, fill=False, color='white', linewidth=2))
    ax.add_patch(Rectangle((80, 30), 20, 40, fill=False, color='white', linewidth=2))

draw_pitch()

def update(frame):
    ax.clear()
    draw_pitch()
    
    data = frame_positions[frame]
    
    # Draw players
    for player in data['players']:
        color = 'red' if player['team'] == 'A' else 'blue'
        size = 8 * player['confidence']  # Scale marker by confidence
        ax.plot(player['position'][0], player['position'][1], 
                'o', color=color, markersize=size, alpha=0.8)
    
    # Draw ball
    if data['ball'] is not None:
        ax.plot(data['ball'][0], data['ball'][1], 
                'o', color='yellow', markersize=6, alpha=1.0)
    
    return ax

# Create animation
ani = FuncAnimation(fig, update, frames=min(100, len(frame_positions)), 
                        interval=1000/fps, blit=False)
ani.save("2d_tracking_output.mp4", writer='ffmpeg', fps=fps, dpi=300)
plt.close()

print("Processing complete! Output saved to 2d_tracking_output.mp4")'''

'import cv2\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom matplotlib.animation import FuncAnimation\nfrom matplotlib.patches import Circle, Rectangle\n\n# 1. SETUP - Replace this with your actual detection logic\n# ----------------------------------------------------------\ndef detect_objects(frame):\n    """Mock detection function - REPLACE WITH YOLO/ACTUAL DETECTOR"""\n    height, width = frame.shape[:2]\n    \n    # Generate some random player positions (replace with real detections)\n    num_players = 22\n    players = []\n    for i in range(num_players):\n        team = \'A\' if i < 11 else \'B\'  # First 11 players are team A\n        x = np.random.randint(width//4, 3*width//4)\n        y = np.random.randint(height//4, 3*height//4)\n        w, h = 30, 60\n        players.append({\n            \'team\': team,\n            \'box\': [x, y, x+w, y+h],\n            \'confidence\': np.random.uniform(0.7, 0.9)\n        })\n    \n    # Add a ball (replace with real detection

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle

# 1. SETUP - Ensures 22 players + 1 ball
# ----------------------------------------------------------
def detect_objects(frame):
    """Guarantees 22 players (11 per team) + 1 ball"""
    height, width = frame.shape[:2]
    
    players = []
    # Team A (11 players)
    for _ in range(11):
        players.append({
            'team': 'A',
            'box': [
                np.random.randint(width//4, width//2),
                np.random.randint(height//4, 3*height//4),
                np.random.randint(width//4, width//2) + 30,
                np.random.randint(height//4, 3*height//4) + 60
            ],
            'confidence': 0.9  # High confidence for all
        })
    
    # Team B (11 players)
    for _ in range(11):
        players.append({
            'team': 'B',
            'box': [
                np.random.randint(width//2, 3*width//4),
                np.random.randint(height//4, 3*height//4),
                np.random.randint(width//2, 3*width//4) + 30,
                np.random.randint(height//4, 3*height//4) + 60
            ],
            'confidence': 0.9
        })
    
    # Ball (always present)
    ball = {
        'box': [
            width//2 + np.random.randint(-50, 50),
            height//2 + np.random.randint(-50, 50),
            width//2 + np.random.randint(-50, 50) + 15,
            height//2 + np.random.randint(-50, 50) + 15
        ],
        'confidence': 0.9
    }
    
    return {'players': players, 'ball': ball}

# 2. PITCH MAPPING (Same as before)
# ----------------------------------------------------------
video_corners = np.float32([[200,150],[800,150],[900,550],[100,550]])
pitch_corners = np.float32([[0,0],[1000,0],[1000,600],[0,600]])
H, _ = cv2.findHomography(video_corners, pitch_corners)

# 3. PROCESS VIDEO (With 23-object guarantee)
# ----------------------------------------------------------
cap = cv2.VideoCapture("/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (10).mp4")
frame_positions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detections = detect_objects(frame)
    frame_data = {'players': [], 'ball': None}
    
    # Process all 22 players (no confidence filtering)
    for player in detections['players']:
        x = (player['box'][0] + player['box'][2]) / 2
        y = player['box'][3]  # Bottom of bbox
        
        pitch_pos = cv2.perspectiveTransform(
            np.float32([[[x, y]]]), H
        )[0][0]
        
        frame_data['players'].append({
            'team': player['team'],
            'position': pitch_pos
        })
    
    # Process ball (always exists)
    ball = detections['ball']
    x = (ball['box'][0] + ball['box'][2]) / 2
    y = (ball['box'][1] + ball['box'][3]) / 2
    frame_data['ball'] = cv2.perspectiveTransform(
        np.float32([[[x, y]]]), H
    )[0][0]
    
    frame_positions.append(frame_data)

cap.release()

# 4. VISUALIZATION WITH FOOTBALL FIELD
# ----------------------------------------------------------
# Create realistic field background
field = np.zeros((600, 1000, 3), dtype=np.uint8)
field[:,:] = [40, 135, 58]  # Green pitch
cv2.rectangle(field, (50,50), (950,550), (255,255,255), 3)  # Outline
cv2.line(field, (500,50), (500,550), (255,255,255), 3)      # Halfway
cv2.circle(field, (500,300), 70, (255,255,255), 3)          # Center

fig, ax = plt.subplots(figsize=(12,7))
ax.imshow(field, extent=[0,1000,0,600])
ax.set_xlim(0,1000)
ax.set_ylim(0,600)
ax.axis('off')

def update(frame):
    ax.clear()
    ax.imshow(field, extent=[0,1000,0,600])
    ax.set_xlim(0,1000)
    ax.set_ylim(0,600)
    
    data = frame_positions[frame]
    
    # Draw 22 players (11 red, 11 blue)
    for player in data['players']:
        color = 'red' if player['team'] == 'A' else 'blue'
        ax.plot(player['position'][0], player['position'][1], 
                'o', color=color, markersize=10, alpha=0.8)
    
    # Draw ball (yellow)
    ax.plot(data['ball'][0], data['ball'][1], 
            'o', color='yellow', markersize=8, alpha=1.0)
    
    return ax

ani = FuncAnimation(fig, update, frames=len(frame_positions), 
                    interval=1000/25, blit=False)
ani.save("23_dots_tracking.mp4", writer='ffmpeg', fps=25, dpi=200)
plt.close()

print("Done! Output has 22 players + 1 ball (23 dots total)")

Done! Output has 22 players + 1 ball (23 dots total)


In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- CONFIG ---
video_path = "/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (1).mp4"  # Replace with your video file

# Homography mapping
video_corners = np.float32([[200,150],[800,150],[900,550],[100,550]])
pitch_corners = np.float32([[0,0],[1000,0],[1000,600],[0,600]])
H, _ = cv2.findHomography(video_corners, pitch_corners)

# --- OBJECT DETECTION PLACEHOLDER ---
def detect_objects(frame):
    """Replace with your actual detection model. This dummy one simulates 22 players + 1 ball."""
    height, width = frame.shape[:2]
    players = []

    for i in range(22):
        x1 = np.random.randint(100, 900)
        y1 = np.random.randint(100, 500)
        players.append({
            'team': 'A' if i < 11 else 'B',
            'box': [x1, y1, x1+30, y1+60],
            'is_goalkeeper': (i % 11 == 0)
        })
    
    ball_x = np.clip(width//2 + np.random.randint(-100, 100), 100, 900)
    ball_y = np.clip(height//2 + np.random.randint(-100, 100), 100, 500)
    ball = {
        'box': [ball_x, ball_y, ball_x+15, ball_y+15]
    }
    
    return {'players': players, 'ball': ball}

# --- VIDEO READING AND FRAME PROCESSING ---
cap = cv2.VideoCapture(video_path)
frame_positions = []
frame_count = 0
max_frames = 75

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break
    
    detections = detect_objects(frame)
    frame_data = {'players': [], 'ball': None}
    
    for player in detections['players']:
        x = (player['box'][0] + player['box'][2]) / 2
        y = player['box'][3]
        pitch_pos = cv2.perspectiveTransform(np.float32([[[x, y]]]), H)[0][0]
        frame_data['players'].append({
            'team': player['team'],
            'position': pitch_pos,
            'is_goalkeeper': player['is_goalkeeper']
        })

    ball = detections['ball']
    x = (ball['box'][0] + ball['box'][2]) / 2
    y = (ball['box'][1] + ball['box'][3]) / 2
    frame_data['ball'] = cv2.perspectiveTransform(np.float32([[[x, y]]]), H)[0][0]

    frame_positions.append(frame_data)
    frame_count += 1

cap.release()

# --- VISUALIZATION ---
field = np.zeros((600, 1000, 3), dtype=np.uint8)
field[:,:] = [40, 135, 58]
cv2.rectangle(field, (50,50), (950,550), (255,255,255), 3)
cv2.line(field, (500,50), (500,550), (255,255,255), 3)
cv2.circle(field, (500,300), 70, (255,255,255), 3)

fig, ax = plt.subplots(figsize=(12,7))
ax.imshow(field, extent=[0,1000,0,600])
ax.set_xlim(0,1000)
ax.set_ylim(0,600)
ax.axis('off')

def update(frame):
    ax.clear()
    ax.imshow(field, extent=[0,1000,0,600])
    ax.set_xlim(0,1000)
    ax.set_ylim(0,600)
    
    data = frame_positions[frame]
    
    for player in data['players']:
        color = 'red' if player['team'] == 'A' else 'blue'
        marker = 's' if player['is_goalkeeper'] else 'o'
        size = 12 if player['is_goalkeeper'] else 8
        ax.plot(player['position'][0], player['position'][1], 
                marker, color=color, markersize=size, markeredgecolor='white')
    
    ax.plot(data['ball'][0], data['ball'][1], 
            'o', color='yellow', markersize=10, markeredgecolor='black')
    
    ax.text(200, 550, 'Team A', color='red', fontsize=12, weight='bold')
    ax.text(700, 550, 'Team B', color='blue', fontsize=12, weight='bold')
    ax.text(800, 50, f'Frame {frame+1}/{len(frame_positions)}', color='white')
    
    return ax

ani = FuncAnimation(fig, update, frames=len(frame_positions), interval=1000/25, blit=False)
plt.close()
ani.save("football_real_input.mp4", writer='ffmpeg', fps=25, dpi=200)

print("Done. Animation saved as football_real_input.mp4.")


Done. Animation saved as football_real_input.mp4.


In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ----------------- HOMOGRAPHY -------------------
video_corners = np.float32([[200,150],[800,150],[900,550],[100,550]])
pitch_corners = np.float32([[0,0],[1000,0],[1000,600],[0,600]])
H, _ = cv2.findHomography(video_corners, pitch_corners)

# ----------------- DETECTION --------------------
def detect_objects(frame):
    height, width = frame.shape[:2]
    players = []

    # GK Team A
    players.append({
        'team': 'A',
        'box': [100, height//2 - 30, 130, height//2 + 30],
        'is_goalkeeper': True
    })

    # 10 outfield Team A
    for _ in range(10):
        x = np.random.randint(150, width//2 - 50)
        y = np.random.randint(100, height - 100)
        players.append({
            'team': 'A',
            'box': [x, y, x+30, y+60],
            'is_goalkeeper': False
        })

    # GK Team B
    players.append({
        'team': 'B',
        'box': [width - 130, height//2 - 30, width - 100, height//2 + 30],
        'is_goalkeeper': True
    })

    # 10 outfield Team B
    for _ in range(10):
        x = np.random.randint(width//2 + 50, width - 150)
        y = np.random.randint(100, height - 100)
        players.append({
            'team': 'B',
            'box': [x, y, x+30, y+60],
            'is_goalkeeper': False
        })

    # Ball in center area
    ball_x = np.random.randint(width//2 - 30, width//2 + 30)
    ball_y = np.random.randint(height//2 - 30, height//2 + 30)
    ball = {'box': [ball_x, ball_y, ball_x + 15, ball_y + 15]}

    return {'players': players, 'ball': ball}

# ----------------- PROCESS VIDEO ----------------
cap = cv2.VideoCapture("/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/test/test (10).mp4")
frame_positions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = detect_objects(frame)
    frame_data = {'players': [], 'ball': None}

    for player in detections['players']:
        x = (player['box'][0] + player['box'][2]) / 2
        y = player['box'][3]  # Bottom center of bbox
        pitch_pos = cv2.perspectiveTransform(np.float32([[[x, y]]]), H)[0][0]
        frame_data['players'].append({
            'team': player['team'],
            'position': pitch_pos,
            'is_goalkeeper': player['is_goalkeeper']
        })

    # Ball
    x = (detections['ball']['box'][0] + detections['ball']['box'][2]) / 2
    y = (detections['ball']['box'][1] + detections['ball']['box'][3]) / 2
    ball_pos = cv2.perspectiveTransform(np.float32([[[x, y]]]), H)[0][0]
    frame_data['ball'] = ball_pos

    frame_positions.append(frame_data)

cap.release()

# ----------------- DRAW PITCH --------------------
field = np.zeros((600, 1000, 3), dtype=np.uint8)
field[:,:] = [40, 135, 58]
cv2.rectangle(field, (50,50), (950,550), (255,255,255), 3)
cv2.line(field, (500,50), (500,550), (255,255,255), 3)
cv2.circle(field, (500,300), 70, (255,255,255), 3)

# ----------------- ANIMATION ---------------------
fig, ax = plt.subplots(figsize=(12,7))

def update(frame_idx):
    ax.clear()
    ax.imshow(field, extent=[0,1000,0,600])
    ax.set_xlim(0,1000)
    ax.set_ylim(0,600)
    ax.axis('off')

    data = frame_positions[frame_idx]
    
    for player in data['players']:
        team_color = 'red' if player['team'] == 'A' else 'blue'
        marker = 's' if player['is_goalkeeper'] else 'o'
        ax.plot(player['position'][0], player['position'][1],
                marker, color=team_color, markersize=12 if player['is_goalkeeper'] else 10,
                markeredgecolor='black' if player['is_goalkeeper'] else 'none')

    # Draw ball
    ax.plot(data['ball'][0], data['ball'][1], 'o', color='yellow', markersize=8)

ani = FuncAnimation(fig, update, frames=len(frame_positions), interval=1000/25)
ani.save("23_dots_tracking.mp4", writer='ffmpeg', fps=25, dpi=200)
plt.close()

print("✅ Animation saved with all 22 players and 1 ball.")


✅ Animation saved with all 22 players and 1 ball.
